# Task 1: Implementing and Experimenting with Randomized Hill CLimbing

## Function Definitions


### Our objective function, f
$f(x,y)= -(y+47)⋅\sin{\sqrt{|\frac{x}{2}+(y+47)|}}-x⋅\sin{\sqrt{|x-(y+47)|}}$

In [ ]:
def f(x,y):
  import math

  # guard to enforce range restrictions
  if x < -512 or x > 512:
    print(f"x = {x} which is outside of bounds [-512,512]")
    return
  if y < -512 or y > 512:
    print(f"y = {y} which is outside of bounds [-512,512]")
    return

  return -1*(y+47)*math.sin( (abs( (x/2) + (y + 47) ) )**0.5 ) - x*math.sin( (abs( x - (y + 47) ))**0.5 )

time: 4.41 ms (started: 2022-02-17 02:53:17 +00:00)


#### Here we check that the function is written correctly.
Based on Steve's post in teams the function should have the following evaluations: <br /> 
* $f(404, 504)= -234.0791964837813$
* $f(0, 0.23) = -26.24644115988084$
* $f(-200, 300) = -194.11322741864842$
* $f(412,-99.9) = -181.75129808649814$


In [ ]:
print(f"f(404,504) = {f(404,504)}")
print(f"f(0,0.23) = {f(0,0.23)}")
print(f"f(-200,300) = {f(-200,300)}")
print(f"f(412,-99.9) = {f(412,-99.9)}")

f(404,504) = -235.0791964837813
f(0,0.23) = -26.24644115988084
f(-200,300) = -194.11322741864842
f(412,-99.9) = -181.75129808649817


Despite the slight discrepancy of the very last digit of the last point's evaluation, overall this confirms the f(x,y) function is written correctly.

### Our Random Hill Climber, RHC
This takes as input parameters:<br />
  * sp: the starting point of the RHC run<br />
  * p: the number of neighbors of the current solution that will be generated in the sample<br />
  * z: the neighbnorhood size<br />
  * seed: a seed value for the random number generator<br /><br />

This returns: <br />
* solutionCounter: the number of solutions that were generated during the run (the total number of points produced in all samplings of neighborhoods) <br />
* (x,y): the best found solution, as a vector (as a tuple here in python) <br />
* f(x,y): the value of this new solution <br />
<br />

The subfunction sampler takes in sp, p, and z as inputs and returns a numpy array of tuples which are 


In [ ]:
def RHC(sp, p, z, seed): # new
  import random
  import numpy as np
  random.seed(seed)
  def sampler(sp, p, z): # this produces our p samples offset by +/- z around sp
    sample = []
    for i in range(p):
      xOffset, yOffset = random.uniform(-z,z), random.uniform(-z,z)
      x,y = sp[0]+xOffset, sp[1]+yOffset

      while x < -512 or x > 512:
        xOffset = random.uniform(-z,z)
        x = sp[0]+xOffset
      while y < -512 or y > 512:
        yOffset = random.uniform(-z,z)
        y = sp[1]+yOffset
    
      sample.append((x,y))
    return np.array(sample)

  solutionCounter = 0
  # this while loop is the heart of the RHC
  x, y = sp[0], sp[1]
  oldBestVal = f(x,y) # the very first start point evaluation
  curBestVal = oldBestVal
  solutionCounter += 1
  while True:
    sample = sampler((x,y), p, z)

    for s in sample: # for each point in the sample
      newVal = f(s[0], s[1]) # is f(eachPoint) better than the oldBestVal?
      solutionCounter += 1
      if newVal < curBestVal: # we use < because we're trying to minimize f
        # if it's better we update bestVal, and (x,y)
        curBestVal = newVal        
        x, y = s[0], s[1]
    
    if curBestVal == oldBestVal: # we aren't finding better values any more, terminate
      break
    else: oldBestVal = curBestVal

  return solutionCounter, (x,y), curBestVal

## Testing RHC against 32 combinations of input parameters
The 32 combinations of input parameters will be broken into 4 sets of 8 combinations, 1 set for each combination of p and z inputs.<br /><br />

The starting points iterated through are as follows:<br />
(-200, 300)<br />
(0, 0.23)<br />
(404, 504)<br />
(412, -99.9)<br />

Each "run1" will have a seed value of 1, and each "run2" will have a seed value of 2.



### some basic setup

In [ ]:
import pandas as pd # will be used to display tables
import numpy as np
import math
!pip install ipython-autotime
%load_ext autotime

time: 215 µs (started: 2022-02-17 02:52:31 +00:00)


In [ ]:
startingPoints = np.array([(-200,300),(0,0.23),(404,504),(412,-99.9)])
seeds = [1,2]

time: 1.74 ms (started: 2022-02-17 02:53:09 +00:00)


### Table 1: z = 0.5, p = 30

In [ ]:
run1 = pd.DataFrame(columns=["startingPoint","#solSearched","sol","f(sol)"])
for sp in startingPoints:
  run1.loc[len(run1.index)] = [sp,*list(RHC(sp,30,0.5,seeds[0]))]
run1

,startingPoint,#solSearched,sol,f(sol)
0,"[-200.0, 300.0]",3151,"(-242.91913880836157, 274.33944723793724)",-559.786274
1,"[0.0, 0.23]",1171,"(8.527950694234312, 15.621670186491828)",-66.843301
2,"[404.0, 504.0]",3931,"(347.2934334493836, 499.3280929227168)",-888.947460
3,"[412.0, -99.9]",3781,"(361.76696942598784, -106.84295744185859)",-419.310069


time: 122 ms (started: 2022-02-15 23:30:03 +00:00)


In [ ]:
run2 = pd.DataFrame(columns=["startingPoint","#solSearched","sol","f(sol)"])
for sp in startingPoints:
  run2.loc[len(run2.index)] = [sp,*list(RHC(sp,30,0.5,seeds[1]))]
run2

,startingPoint,#solSearched,sol,f(sol)
0,"[-200.0, 300.0]",3151,"(-242.9635492731161, 274.36665574282694)",-559.786803
1,"[0.0, 0.23]",1111,"(8.49006885389648, 15.58728032769306)",-66.843147
2,"[404.0, 504.0]",3901,"(347.25077306011787, 499.3917713548673)",-888.948158
3,"[412.0, -99.9]",3931,"(361.65461985082345, -106.95614824841991)",-419.311904


time: 209 ms (started: 2022-02-15 23:28:31 +00:00)


In [ ]:
df = pd.DataFrame(columns = ["Run1","Run2"])
df

,Run1,Run2


time: 16.6 ms (started: 2022-02-15 23:32:06 +00:00)


### Table 2: z = 0.5, p = 250

In [ ]:
run1 = pd.DataFrame(columns=["startingPoint","#solSearched","sol","f(sol)"])
for sp in startingPoints:
  run1.loc[len(run1.index)] = [sp,*list(RHC(sp,250,0.5,seeds[0]))]
run1

,startingPoint,#solSearched,sol,f(sol)
0,"[-200.0, 300.0]",22751,"(-242.9618565745455, 274.3680869924391)",-559.786806
1,"[0.0, 0.23]",8251,"(8.427082079848402, 15.661571984646836)",-66.843647
2,"[404.0, 504.0]",30251,"(347.319556750075, 499.39448546659435)",-888.949028
3,"[412.0, -99.9]",28001,"(361.65304368534623, -106.95035551491624)",-419.311863


time: 792 ms (started: 2022-02-15 23:28:31 +00:00)


In [ ]:
run2 = pd.DataFrame(columns=["startingPoint","#solSearched","sol","f(sol)"])
for sp in startingPoints:
  run2.loc[len(run2.index)] = [sp,*list(RHC(sp,250,0.5,seeds[1]))]
run2

,startingPoint,#solSearched,sol,f(sol)
0,"[-200.0, 300.0]",23001,"(-242.96899230618962, 274.39525127339084)",-559.786775
1,"[0.0, 0.23]",8501,"(8.426507742632268, 15.646125147160918)",-66.843636
2,"[404.0, 504.0]",29751,"(347.33592725257216, 499.4152830201381)",-888.949105
3,"[412.0, -99.9]",28001,"(361.6892569202703, -106.96825354508435)",-419.311870


time: 662 ms (started: 2022-02-15 23:28:32 +00:00)


### Table 3: z = 3, p = 30

In [ ]:
run1 = pd.DataFrame(columns=["startingPoint","#solSearched","sol","f(sol)"])
for sp in startingPoints:
  run1.loc[len(run1.index)] = [sp,*list(RHC(sp,30,3,seeds[0]))]
run1

,startingPoint,#solSearched,sol,f(sol)
0,"[-200.0, 300.0]",571,"(-243.25082941852483, 274.556615126738)",-559.775527
1,"[0.0, 0.23]",241,"(8.833039038237569, 15.977849850613351)",-66.811637
2,"[404.0, 504.0]",691,"(347.33546383861824, 499.1206459413417)",-888.920919
3,"[412.0, -99.9]",691,"(361.53709832280276, -107.27807228714481)",-419.299514


time: 87.5 ms (started: 2022-02-15 23:28:33 +00:00)


In [ ]:
run2 = pd.DataFrame(columns=["startingPoint","#solSearched","sol","f(sol)"])
for sp in startingPoints:
  run2.loc[len(run2.index)] = [sp,*list(RHC(sp,30,3,seeds[1]))]
run2

,startingPoint,#solSearched,sol,f(sol)
0,"[-200.0, 300.0]",601,"(-242.64223221368846, 274.35828570602325)",-559.775308
1,"[0.0, 0.23]",241,"(8.44401110476838, 15.459105193997551)",-66.838278
2,"[404.0, 504.0]",721,"(347.6726438164906, 499.51087506513426)",-888.928551
3,"[412.0, -99.9]",751,"(361.4572051622431, -107.18526527867137)",-419.305359


time: 86.3 ms (started: 2022-02-15 23:28:33 +00:00)


### Table 4: z = 3, p = 250

In [ ]:
run1 = pd.DataFrame(columns=["startingPoint","#solSearched","sol","f(sol)"])
for sp in startingPoints:
  run1.loc[len(run1.index)] = [sp,*list(RHC(sp,250,3,seeds[0]))]
run1

,startingPoint,#solSearched,sol,f(sol)
0,"[-200.0, 300.0]",4001,"(-242.87017795912706, 274.2606404588166)",-559.783161
1,"[0.0, 0.23]",1751,"(8.532092912039467, 15.631654797984885)",-66.843295
2,"[404.0, 504.0]",5251,"(347.55850008094933, 499.5267780446654)",-888.941053
3,"[412.0, -99.9]",5251,"(361.4863218602253, -107.08684874777252)",-419.308770


time: 189 ms (started: 2022-02-15 23:28:33 +00:00)


In [ ]:
run2 = pd.DataFrame(columns=["startingPoint","#solSearched","sol","f(sol)"])
for sp in startingPoints:
  run2.loc[len(run2.index)] = [sp,*list(RHC(sp,250,3,seeds[1]))]
run2

,startingPoint,#solSearched,sol,f(sol)
0,"[-200.0, 300.0]",4251,"(-243.2801986121297, 274.4650073090307)",-559.777725
1,"[0.0, 0.23]",2251,"(8.493298812498152, 15.730087276476599)",-66.842575
2,"[404.0, 504.0]",5251,"(347.27683223032085, 499.4324167309424)",-888.948131
3,"[412.0, -99.9]",5001,"(361.57841506762026, -106.95918353472823)",-419.310909


time: 158 ms (started: 2022-02-15 23:43:14 +00:00)


## The "33rd" run

Here I'll use restarts to try and brute force my way to a better solution than ~ -888.95. <br />

We can use low p and high z with many restarts to find ballpark area of good minima 'quickly', then crank up p and lower z along with that ballpark area as a start point to find an even better minima. 

This can be done in two different ways:

In [ ]:
def gridRestartRHC(dist, p1, p2, z2, seed): #dist is the spacing between the starting points in the starting point grid
#p1,z1=dist/2 are the parameters used for the crude discovery
#p2,z2 are the fine-grain search parameters
  # low p and high z with many restarts to find ballpark area of good minimum
    #first we produce the grid
  sequence = [x for x in range(-512,513,dist)]
  allPerms = []
  for i in sequence:
    for j in sequence:
      allPerms.append((i,j))
  print(f"We have {len(allPerms)} starting points in the grid.")
    # now we use the grid as starting points for a crude search
  best = None
  totalSolutionsInspected = 0
  for sp in allPerms:
    newRun = RHC(sp,p1,dist/2,seed) #'z1' is based on dist
    totalSolutionsInspected += newRun[0]
    if best == None:
      best = newRun
    elif newRun[2] < best[2]:
        best = newRun
  print(f"Intermediate best was:\nf({best[1][0]},{best[1][1]}) = {best[2]}")

  # next we crank up p and lower z to perform a higher resolution search starting at the intermediate best to further optimize
  best = RHC(best[1],p2,z2,seed)
  totalSolutionsInspected += best[0]
  # finally we print our results
  print(f"After searching through {totalSolutionsInspected} total points, the best solution found was:\nf({best[1][0]},{best[1][1]} = {best[2]}")

time: 39.7 ms (started: 2022-02-16 23:16:03 +00:00)


In [ ]:
gridRestartRHC(2**3,10,500,0.1,1)

We have 16641 starting points
intermediate best is f(511.9841390251008,404.154124433487) = -959.5823445692604
After searching through 3388272, the best solution found was:
f(511.999705293255,404.2221969380265 = -959.6395656999875
time: 24.6 s (started: 2022-02-16 22:54:01 +00:00)


In [ ]:
gridRestartRHC(2**4,10,500,0.1,2)

We have 4225 starting points
intermediate best is f(511.83850516287004,403.96302456135834) = -959.072139937412
After searching through 459026, the best solution found was:
f(511.99963403809363,404.2124499520243 = -959.6390121804029
time: 3.74 s (started: 2022-02-16 23:09:50 +00:00)


In [ ]:
gridRestartRHC(2**5,10,500,0.1,4)

We have 1089 starting points
intermediate best is f(511.83906287362146,402.75248921906297) = -957.0839315471355
After searching through 72390, the best solution found was:
f(511.999761775651,404.2070049534318 = -959.6391690723226
time: 639 ms (started: 2022-02-16 23:10:09 +00:00)


In [ ]:
gridRestartRHC(2**6,10,500,0.1,1)

We have 289 starting points in the grid.
Intermediate best was:
f(508.5047804195073,398.1785620698101) = -936.3503624725432
After searching through 44060 total points, the best solution found was:
f(511.9999805921124,404.22042617259797 = -959.6404502773961
time: 344 ms (started: 2022-02-16 23:16:38 +00:00)


In [ ]:
gridRestartRHC(2**7,10,500,0.1,3)

We have 81 starting points in the grid.
Intermediate best was:
f(509.43898527929935,397.4003434125703) = -927.3338278529279
After searching through 38862 total points, the best solution found was:
f(511.99991152608544,404.24646397101253 = -959.6401162282449
time: 479 ms (started: 2022-02-16 23:16:22 +00:00)


In [ ]:
gridRestartRHC(2**8,10,500,0.1,2)

We have 25 starting points
intermediate best is f(474.5694663274505,425.26327124582883) = -945.1525059005091
After searching through 42716, the best solution found was:
f(482.34744491578766,432.8743443595374 = -956.9181911201148
time: 339 ms (started: 2022-02-16 22:54:31 +00:00)


In [ ]:
gridRestartRHC(2**9,10,500,0.1,2)

We have 9 starting points
intermediate best is f(499.3074712828409,388.22638771175394) = -868.4027505879723
After searching through 84250, the best solution found was:
f(511.99975327129636,404.2360046765701 = -959.6398051777426
time: 571 ms (started: 2022-02-15 23:29:06 +00:00)


In [ ]:
gridRestartRHC(2**10,10,500,0.1,3)

We have 4 starting points
intermediate best is f(331.27572389230977,491.72259885861547) = -851.1541984110589
After searching through 82105, the best solution found was:
f(347.3268822436294,499.41656617595163 = -888.9491248028082
time: 598 ms (started: 2022-02-16 22:55:45 +00:00)


In [ ]:
gridRestartRHC(2**11,10,500,0.1,2)

We have 1 starting points
intermediate best is f(290.3848096323327,-459.258328788934) = -529.0910440204091
After searching through 142522, the best solution found was:
f(283.07560618133476,-487.1268655768635 = -718.1674592435128
time: 999 ms (started: 2022-02-16 22:55:34 +00:00)


## Effects of changing p

In [ ]:
changingP = pd.DataFrame(columns = ["p","#solSearched","sol","f(sol)"])
for i in range(6):
  changingP.loc[len(changingP.index)] = [2**(5+i), *RHC((0,0),2**(5+i),1,1)]
changingP

,p,#solSearched,sol,f(sol)
0,32,641,"(8.368065225978475, 15.769662263777697)",-66.841578
1,64,1281,"(8.424355140385568, 15.661582803381881)",-66.843636
2,128,2305,"(8.469027518040418, 15.67033483913617)",-66.843641
3,256,4609,"(8.428600597390634, 15.66023501564695)",-66.843655
4,512,9217,"(8.43905188351823, 15.661658491971366)",-66.843685
5,1024,17409,"(8.438538441793161, 15.638006949365966)",-66.843656


time: 318 ms (started: 2022-02-16 22:13:37 +00:00)


## Effects of changing z

In [ ]:
changingZ = pd.DataFrame(columns = ["z","#solSearched","sol","f(sol)"])
for i in range(10):
  changingZ.loc[len(changingZ.index)] = [-0.5+2**i, *RHC(sp=(0,0),p=128,z=-0.5+2**i,seed=1)]
changingZ

,z,#solSearched,sol,f(sol)
0,0.5,4481,"(8.433646406913356, 15.657116980337639)",-66.843677
1,1.5,1537,"(8.431236056667938, 15.695959924907129)",-66.843428
2,3.5,769,"(8.277138574925646, 15.895478714784103)",-66.834728
3,7.5,641,"(8.327784577285831, 15.371997647823882)",-66.829500
4,15.5,385,"(7.181467549434279, 16.200865349733018)",-66.716769
5,31.5,385,"(-45.76572979797494, 39.67202300311018)",-125.792416
6,63.5,385,"(-53.761820819446584, 37.717206845914426)",-120.621349
7,127.5,769,"(423.98059702924866, 181.965986500953)",-609.391304
8,255.5,385,"(-442.58572380749183, -374.06179004995755)",-752.744422
9,511.5,257,"(-454.08586663628256, 378.5203887457026)",-871.182142


time: 146 ms (started: 2022-02-16 22:25:09 +00:00)


In [ ]:
type(RHC((0,0),1,1,1))

tuple

time: 33.7 ms (started: 2022-02-16 22:11:11 +00:00)


## Interpretation
There are two competing motivations in choosing p and z - one is our desire to find the lowest point in the space space of f, the other is to obtain results quickly. Unfortunately, as with so many things, there is a tradeoff between these two motivations. Selecting a low p and high z will result in an algorithm that terminates relatively quickly, however it may not actually find a local optimum. On the other hand, choosing a high p and a low z will, in general, find a better local optimum but it will take much longer to converge thereupon. It is therefore necessary to find a balance such that a good solution is found in a relatively quick time.

Ultimately, however, the best value of f found is mostly determined by the starting point, rather than by p or z, which instead merely affect the speed at which a particular approximation of a local optimum is found. This is precisely why restarts are so widely used alongside hill climbing, to expose the algorithm to different peaks and troughs to explore, rather than simply exploring the same peak or trough at different rates.